In [6]:
# 生成train_data13.csv
# 添加规则匹配的训练集订单（训练集路径不包含测试集路径，将匹配的训练集和测试集拼接）

import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

def process_order(df):
    order_df = pd.read_csv( '../../../dataset/order_data/B_order_info.csv')
    trace_df = order_df.drop_duplicates(['TRANSPORT_TRACE']).reset_index(drop = True)
    
    df = df.merge(trace_df[['TRANSPORT_TRACE', 'start_lon', 'start_lat', 'end_lon', 'end_lat']], on = 'TRANSPORT_TRACE', how = 'left')
    
    train_data = pd.DataFrame()
    orders = df.loadingOrder.unique()
    print('orders num:{}'.format(len(orders)))
    for i in range(len(orders)):
        order_df = df[df['loadingOrder'] == orders[i]].reset_index(drop = True)
        start_lon = order_df.loc[0, 'start_lon']
        start_lat = order_df.loc[0, 'start_lat']
        
        end_lon = order_df.loc[0, 'end_lon']
        end_lat = order_df.loc[0, 'end_lat']
        
#         print('\nstart_lon:{}\tstart_lat:{}'.format(start_lon, start_lat))
#         print('end_lon:{}\tend_lat:{}'.format(end_lon, end_lat))
        
        dis_start, dis_end = [], []
        for j in range(len(order_df)):
            dis1 = abs(order_df.loc[j, 'longitude'] - start_lon) + abs(order_df.loc[j, 'latitude'] - start_lat)
            dis2 = abs(order_df.loc[j, 'longitude'] - end_lon) + abs(order_df.loc[j, 'latitude'] - end_lat)
        
            dis_start.append(dis1)
            dis_end.append(dis2)
        
        index_start = dis_start.index(min(dis_start))
        index_end = dis_end.index(min(dis_end))
        
        print('order{}:{} trace:{}\t start:{}\tend:{}'.format(i, orders[i], order_df.loc[0, 'TRANSPORT_TRACE'], index_start, index_end))
        
        if (index_end - index_start > 2):
            order_df = order_df.iloc[index_start:index_end + 1, :].reset_index(drop = True)
            train_data = train_data.append(order_df, ignore_index = True)
    
    train_data = train_data.drop(['start_lon', 'start_lat', 'end_lon', 'end_lat'], axis = 1)
    train_data = train_data.sort_values(['loadingOrder', 'timestamp'], ascending = True).reset_index(drop = True)
    return train_data


# 将匹配的重复训练集订单更名添加
def get_match_df(train_df, match_order_df):
    # 测试集处理
    test_df = pd.read_csv( '../../../dataset/test_data/B_testData0807.csv')
    test_df['vesselNextport'] = train_df.loc[0, 'vesselNextport']
    test_df['vesselNextportETA'] = train_df.loc[0, 'vesselNextportETA']
    test_df['vesselStatus'] = train_df.loc[0, 'vesselStatus']
    test_df['vesselDatasource'] = train_df.loc[0, 'vesselDatasource']
    test_df = test_df.drop(['onboardDate'], axis = 1)
    test_df = test_df[train_df.columns]

     # 测试集清洗
    test_df = test_df[test_df['direction'] != -1]

    # 清洗空值记录
    test_df = test_df.dropna(how = 'any', axis = 0).reset_index(drop = True)
    print('test_num:{}'.format(len(test_df)))

    train_data = pd.DataFrame()
    for i in range(len(match_order_df)):
        print('\ni:{}'.format(i))
        match_order = match_order_df.loc[i, 'match_train_order']
        
        # 训练集数据
        match_df = train_df[train_df['loadingOrder'] == match_order] .reset_index(drop = True)
        
        # 测试集数据
        match_test = test_df[test_df['loadingOrder'] == match_order_df.loc[i, 'loadingOrder']]

        match_df = match_df.append(match_test, ignore_index = True).reset_index(drop = True)
        match_df['loadingOrder'] = match_order + '_' + '{}'.format(i)
        match_df['TRANSPORT_TRACE'] =  match_order_df.loc[i, 'TRANSPORT_TRACE']
    
        match_df = match_df.drop_duplicates(['timestamp']).reset_index(drop = True)
    
        print(match_order_df.loc[i, 'TRANSPORT_TRACE'])
        print(match_df.loc[0,'TRANSPORT_TRACE'])
        
        train_data = train_data.append(match_df, ignore_index = True)

    train_data = train_data[train_df.columns]
    train_data = train_data.sort_values(['loadingOrder', 'timestamp'], ascending = True)
    
    return train_data



start = time.process_time()

# 拼接订单
match_order_df = pd.read_csv( '../../../dataset/test_data/joint_order.csv')
match_order_df = match_order_df[match_order_df['match_train_order'] != 'UM418735113516'].reset_index(drop = True)
match_orders = match_order_df.match_train_order.unique()

add_df = pd.DataFrame()
for i in range(10):
    print('i:{}'.format(i))
    train_df = pd.read_csv('../../../dataset/chunk_data/train_data{}.csv'.format(i), header = None)
    train_df.columns = ['loadingOrder', 'carrierName', 'timestamp', 'longitude',
                          'latitude', 'vesselMMSI', 'speed', 'direction', 'vesselNextport',
                          'vesselNextportETA', 'vesselStatus', 'vesselDatasource', 'TRANSPORT_TRACE']
    print(train_df.loc[len(train_df)-1, 'loadingOrder'])
    
    train_df = train_df.loc[(train_df['loadingOrder'].isin (match_orders))]
    train_df = train_df[(train_df['direction'] <= 36000) & (train_df['speed'] < 60)].reset_index(drop = True)
    
    add_df = add_df.append(train_df, ignore_index = True)
    print(len(train_df))
    print('read csv: {} seconds'.format(time.process_time() - start))

add_df = add_df[train_df.columns]
add_df = add_df.sort_values(['loadingOrder', 'timestamp'], ascending = True)


# 将匹配的重复训练集订单更名添加
add_df = get_match_df(add_df, match_order_df)

# 截取匹配路径
# add_df = process_order(add_df)

orders = add_df.loadingOrder.unique()
print(orders)
print(len(orders))
print(add_df.TRANSPORT_TRACE.unique())

add_df.to_csv('../../../dataset/train_data/train_wash13.csv', index = False, header = None)
print(add_df)
print('running time: {} seconds'.format(time.process_time() - start))

i:0
CX425495580514
12395
read csv: 22.274491065000007 seconds
i:1
FK857864540293
24764
read csv: 43.28202000900001 seconds
i:2
HW907489849828
10896
read csv: 64.025428702 seconds
i:3
KP455371889804
3399
read csv: 84.16650778400003 seconds
i:4
NF901306911140
26271
read csv: 104.47163692200002 seconds
i:5
PV603106113253
21853
read csv: 124.81980713100006 seconds
i:6
SM861453907016
12665
read csv: 144.627106927 seconds
i:7
VC489290307158
10938
read csv: 165.51051494700005 seconds
i:8
XV497928381083
8040
read csv: 186.05119028700005 seconds
i:9
ZZ992682575994
8048
read csv: 205.70178636000003 seconds
test_num:56917

i:0
CNSHK-HKHKG-TWKHH-CNNBG-CNSHA-CNTAO-KRPUS-MXZLO-PABLB-PAMIT-COCTG-JMKIN-DOCAU
CNSHK-HKHKG-TWKHH-CNNBG-CNSHA-CNTAO-KRPUS-MXZLO-PABLB-PAMIT-COCTG-JMKIN-DOCAU

i:1
CNSHK-EGPSD
CNSHK-EGPSD

i:2
CNSHK-EGPSD
CNSHK-EGPSD

i:3
CNSHK-KRINC
CNSHK-KRINC

i:4
CNYTN-CAVAN
CNYTN-CAVAN

i:5
CNYTN-GBFXT
CNYTN-GBFXT

i:6
CNSHK-CLSAI
CNSHK-CLSAI

i:7
CNSHK-THLCH
CNSHK-THLCH

i:8
CNYTN-GRPIR
